In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from itertools import combinations 
import csv
import collections
import numpy
import sys
numpy.set_printoptions(threshold=sys.maxsize)
from sklearn.model_selection import train_test_split

In [ ]:
# with open('Output/ratings.dat', encoding='utf8') as dat_file, open('Output/ratings.csv', 'w') as csv_file:
#     csv_writer = csv.writer(csv_file)

#     for line in dat_file:
#         row = [field.strip() for field in line.split('::')]
#         csv_writer.writerow(row)

# ratings = pd.read_csv('Output/ratings.csv', sep=',', encoding='latin-1')
# ratings.columns = ['user_id', 'movie_id', 'rating', 'timestamp']
# print(ratings.sample)

In [ ]:
# ratings_train, ratings_test = train_test_split(ratings, test_size=0.1)

In [ ]:
# print(len(ratings_train))
# print(len(ratings_test))

In [2]:
with open('Output/movies.dat', encoding='utf8') as dat_file, open('Output/movies.csv', 'w') as csv_file:
    csv_writer = csv.writer(csv_file)

    for line in dat_file:
        row = [field.strip() for field in line.split('::')]
        csv_writer.writerow(row)

movies = pd.read_csv('Output/movies.csv', sep=',', encoding='latin-1')
movies.columns = ['movie_id', 'title', 'genre']
#print(movies.sample)

In [3]:
with open('Output/tags.dat', encoding='utf8') as dat_file, open('Output/tags.csv', 'w') as csv_file:
    csv_writer = csv.writer(csv_file)

    for line in dat_file:
        row = [field.strip() for field in line.split('::')]
        csv_writer.writerow(row)

tags = pd.read_csv('Output/tags.csv', sep=',', encoding='latin-1')
tags.columns = ['user_id', 'movie_id', 'tags', 'timestamp']
#print(tags.head)

In [4]:
tags['tags'] = tags['tags'].str.lower()


In [5]:
counts_dict = {}
movies_dict = dict()
tags_list = []

In [6]:
for t in tags['tags'].tolist():
    tags_list.append(t)

In [7]:
for ix in tags.index:
    key = tags.loc[ix]['movie_id']
    if key not in movies_dict:
        movies_dict[key] = []
    movies_dict[key].append(tags.loc[ix]['tags'])

In [8]:
for tag in tags_list:
    counts_dict[tag] = counts_dict.get(tag, 0) + 1

In [9]:
counts_dict_sorted = sorted(counts_dict, key=counts_dict.get, reverse=True)
top_1000_counts = counts_dict_sorted[0:1000]
print(len(top_1000_counts))

1000


In [10]:
with open('Output/top1000counts.txt', 'w') as f:
    print(top_1000_counts, file=f)

In [11]:
with open('Output/moviesdict.txt', 'w') as f:
    print(movies_dict, file=f)

In [12]:
movies_concepts_array = np.zeros(shape=(len(movies_dict), len(top_1000_counts)))
row_number = -1
for movie in movies_dict:
    row_number = row_number + 1
    col_number = -1
    concepts_list = movies_dict[movie]
    for concept in top_1000_counts:
        col_number = col_number + 1
        if concept in concepts_list:
            movies_concepts_array[row_number][col_number] = 1

In [13]:
with open('Output/moviesconceptsarray.txt', 'w') as f:
    print(movies_concepts_array, file=f)
    # for line in movies_concepts_array:
    #     numpy.savetxt(f, line, fmt='%.2f')

In [12]:
# from scipy import spatial

# def cosine_similarity(movie_1, movie_2):
#     movie_num_1 = list(movies_dict).index(movie_1)
#     movie_num_2 = list(movies_dict).index(movie_2)
#     result = 1- spatial.distance.cosine(movies_concepts_array[movie_num_1], movies_concepts_array[movie_num_2])
#     return result

In [13]:
from numpy import dot
from numpy.linalg import norm

def cosine_sim(movie_1, movie_2):
    if movie_1 not in movies_dict or movie_2 not in movies_dict:
        return 0
    movie_num_1 = list(movies_dict).index(movie_1)
    movie_num_2 = list(movies_dict).index(movie_2)
    a = movies_concepts_array[movie_num_1]
    b = movies_concepts_array[movie_num_2]
    if norm(a) == 0 or norm(b) == 0:
        return 0
    cos_sim = dot(a, b)/(norm(a)*norm(b))
    return cos_sim


In [15]:
users_dict = dict()

In [19]:
ratings__grouped = ratings_train.groupby('user_id')

In [20]:
ratings_matrix = np.zeros(shape=(len(ratings_grouped), len(movies_dict)))
key = 0
row = -1
existing_users = []
for ix in ratings_train.index:
    user_id = ratings_train.loc[ix]['user_id']
    if user_id not in existing_users:
        existing_users.append(user_id)
        row = row + 1
        users_dict[user_id] = row

    movie_id = ratings_train.loc[ix]['movie_id']
    ratings_val = ratings_train.loc[ix]['rating']

    if movie_id in movies_dict:
        col = list(movies_dict).index(movie_id)
        ratings_matrix[row][col] = ratings_val

In [ ]:
with open('ratings-matrix.txt', 'w') as f:
    print(ratings_matrix, file=f)

In [24]:
def ratings_prediction(movie_id, user_id):

    if movie_id not in movies_dict:
        return 0
    if user_id not in users_dict:
        return 0
    
    pred_rating = 0

    user_ix = list(users_dict).index(user_id)

    for key, value in movies_dict.items():
        if key == movie_id:
            continue
        else:
            movie_ix = list(movies_dict).index(key)
            movie_sim = cosine_sim(key, movie_id)
            if movie_sim != 0:
                if ratings_small_matrix[user_ix][movie_ix] != 0:
                    print(key)
                pred_rating = pred_rating + movie_sim * ratings_small_matrix[user_ix][movie_ix]
    
    return pred_rating

In [ ]:
predicted_ratings = []
actual_ratings = []
#calculating accuracy
for ix in ratings_test.index:
    test_values = ratings_small_test.loc[ix]
    test_movie_id = test_values['movie_id']
    test_user_id = test_values['user_id']
    test_rating = test_values['rating'] 
    predicted_rating = ratings_prediction(test_movie_id, test_user_id) 
    #print(predicted_rating)
    if predicted_rating != 0:
        predicted_ratings.append(predicted_rating)
        actual_ratings.append(actual_ratings)

In [ ]:
correct = 0
total = 0
for i in range(len(actual_rating_list)):
    if actual_rating_list[i] == round(pred_rating_list[i]):
        correct = correct + 1
    total = total + 1

In [ ]:
#Accuracy
print(correct/total)

In [28]:
# LOW SIMILARITY VALUES DUE TO A LOT OF UNIQUE TAGS 
# print(ratings_prediction(6664, 18))
# tags_list = movies_dict[6664]
# print(tags_list)
# print(cosine_sim(339, 300))

['campy', 'arnold schwarzenegger', 'a legend', 'good action', 'must see!', 'so funny', 'tense', 'actio', 'arnold schwarzenegger', 'mark l. lester', 'campy', 'arnold schwarzenegger', 'a legend', 'good action', 'must see!', 'so funny', 'tense', 'actio', 'arnold schwarzenegger', 'mark l. lester', 'daring rescues', 'earnest', 'forceful', 'gritty', 'menacing', 'race against time', 'rousing', 'tense', 'visceral', 'bad guy dresses gay', 'camp', 'hostage', 'murder', 'plot point:time is short', 'friday night movie']


In [48]:
with open('predicted-ratings.txt', 'w') as f:
    for p in predicted_ratings:
        print(p, file=f)

In [ ]:
with open('actual-ratings.txt', 'w') as f:
    for a in actual_ratings:
        print(a, file=f)